In [251]:
import csv
import numpy as np
import random
from sklearn.svm import SVC

In [361]:
score_list = []
team_list=[]
allteam=set()
test_Y = []
info = []
feature_len = 13
with open('WNCAATourneyDetailedResults.csv') as f:
    reader = csv.reader(f, delimiter=',')
    headers = next(reader, None)
    j=0
    for row in reader:
        if row[0]=='2018':
            allteam.add(row[2])
            allteam.add(row[4])
            if int(row[2])<int(row[4]):
                teams=[row[2],row[4]]
                team_list.append(teams)
                test_Y.append(1)
            else:
                teams=[row[4],row[2]]
                team_list.append(teams)
                test_Y.append(0)
        else:
            row_list =[]
            i = 0
            for score in row:
                if i >7 and i not in info:
                    row_list.append(float(score))
                i=i+1
            if j%2 ==0:
                lose = row_list[feature_len:]
                win = row_list[:feature_len]
                #diff = [lose[i]-win[i] for i in range(13)]
                lose.extend(win)
                #lose.extend(diff)
                lose.append(0)
                score_list.append(lose)
            else:
                lose = row_list[feature_len:]
                win = row_list[:feature_len]
                #diff = [win[i]-lose[i] for i in range(13)]
                #row_list.extend(diff)
                row_list.append(1)
                score_list.append(row_list)
            j = j+1
print(len(score_list))
with open('WRegularSeasonDetailedResults.csv') as f:
    reader = csv.reader(f, delimiter=',')
    headers = next(reader, None)
    j=0
    for row in reader:
        row_list =[]
        i = 0
        #if [row[2],row[4]] in team_list or [row[4],row[2]] in team_list:           
        for score in row:
            if i >7 and i not in info:
                row_list.append(float(score))
            i=i+1
        if j%2 ==0:
            lose = row_list[feature_len:]
            win = row_list[:feature_len]
            #diff = [lose[i]-win[i] for i in range(13)]
            lose.extend(win)
            #lose.extend(diff)
            lose.append(0)
            score_list.append(lose)

        else:
            lose = row_list[feature_len:]
            win = row_list[:feature_len]
            #diff = [win[i]-lose[i] for i in range(13)]
            #row_list.extend(diff)
            row_list.append(1)
            score_list.append(row_list)
        j = j+1
score_list =np.array((score_list))

504


In [362]:
random.shuffle(score_list)
print(len(score_list))

46846


In [308]:
print(allteam)

set(['3355', '3125', '3276', '3328', '3274', '3323', '3299', '3326', '3169', '3294', '3329', '3304', '3437', '3163', '3278', '3301', '3438', '3453', '3384', '3129', '3281', '3211', '3212', '3208', '3343', '3346', '3124', '3257', '3181', '3189', '3166', '3400', '3390', '3141', '3333', '3332', '3263', '3417', '3266', '3311', '3268', '3173', '3261', '3138', '3177', '3280', '3179', '3401', '3393', '3203', '3234', '3397', '3378', '3376', '3114', '3113', '3443', '3370', '3195', '3143', '3110', '3251', '3199', '3273'])


In [342]:
Y = score_list[:,-1]
X = score_list[:,:-1]
#train_X = X[:45000]
#train_Y = Y[:45000]
#valid_X = X[45000:]
#valid_Y = Y[45000:]

In [247]:
from sklearn.svm import SVC
from sklearn import svm
clf_list =[]
C_list =[1e-3]
for i in range(1):
    val_acc_list=[]
    curr_clf=[]
    clf = SVC(kernel='linear',C=C_list[i],probability =True)
    clf.fit(train_X,train_Y)

    test_predictions = clf.predict(valid_X)
    val_acc = [pred == y for pred,y in zip(test_predictions, valid_Y)]
    print('Validation Accuracy for C = '+ str(C_list[i])+ ' classifier ', 
          float(sum(val_acc))/len(valid_Y))
    
    train_predictions = clf.predict(train_X)
    train_acc = [pred == y for pred,y in zip(train_predictions, train_Y)]
    print('Training Accuracy for C = '+ str(C_list[i])+ ' classifier ', 
          float(sum(train_acc))/len(train_Y))

    val_acc_list.append(val_acc)
    clf_list.append(clf)

('Validation Accuracy for C = 0.001 classifier ', 0.998952331063384)
('Training Accuracy for C = 0.001 classifier ', 0.9991111111111111)


In [263]:
team_list=[]
allteam=set()
with open('../WSampleSubmissionStage2.csv') as f:
    reader = csv.reader(f, delimiter='_')
    headers = next(reader, None)
    for row in reader:
        last2 = row[2].split(',')
        team=[row[1],last2[0]]
        allteam.add(row[1])
        allteam.add(last2[0])
        team_list.append(team)

In [363]:
team_detail=set()
team_dict = dict()
team_mean_dict=dict()
with open('WRegularSeasonDetailedResults.csv') as f:
    reader = csv.reader(f, delimiter=',')
    headers = next(reader, None)
    for row in reader:
        if row[0]=='2017':
            team1=row[2]
            team2=row[4]
            if team1 in allteam:
                if team1 not in team_dict:
                    team_dict[team1]=[[int(row[i]) for i in range(8,8+feature_len)]]
                else:
                    team_dict[team1].append([int(row[i]) for i in range(8,8+feature_len)])
            if team2 in allteam:
                if team2 not in team_dict:
                    team_dict[team2]=[[int(row[i]) for i in range(8+feature_len,8+2*feature_len)]]
                else:
                    team_dict[team2].append([int(row[i]) for i in range(8+feature_len,8+2*feature_len)])
for t in allteam:
    team_mean_dict[t]=np.max(np.array(team_dict[t]),axis=0)

In [364]:
test_X =[]
for i,j in team_list:
    current=list(team_mean_dict[i])
    current.extend(team_mean_dict[j])
    test_X.append(current)    

In [365]:
test_X = np.array(test_X)
test_Y = np.array(test_Y)

In [366]:
naive_Y=[1 if test_X[i][0]*2+test_X[i][2]+test_X[i][4]-test_X[i][13]*2-test_X[i][15]-test_X[i][17]>0 
 else 0 for i in range(len(test_Y))]

In [367]:
test_acc = [x == y for x,y in zip(naive_Y, test_Y)]
print(float(sum(test_acc))/63)

0.650793650794


In [355]:
clf = SVC(kernel='linear',C=0.001,probability =True)
clf.fit(X,Y)
clf.score(X,Y)

0.9990180591726081

In [356]:
clf.score(test_X,test_Y)

0.6507936507936508

In [368]:
clf.coef_ 

array([[ 0.77881826,  0.01938597,  0.37927752,  0.00609028,  0.36381826,
         0.0313458 , -0.02961228,  0.09639579,  0.0292662 , -0.04341945,
         0.01798265,  0.01111754, -0.02637719, -0.75828745, -0.04390671,
        -0.41841601,  0.00185671, -0.35790642, -0.03823474,  0.04678375,
        -0.07060259, -0.02257428,  0.04246143, -0.01606106,  0.00430183,
         0.01926071]])

In [369]:
from sklearn.ensemble import RandomForestClassifier
tree_clf = RandomForestClassifier(n_estimators=50,random_state=0)
tree_clf.fit(X,Y)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=50, n_jobs=1,
            oob_score=False, random_state=0, verbose=0, warm_start=False)

In [370]:
print(tree_clf.score(X,Y))
print(tree_clf.score(test_X,test_Y))

1.0
0.6825396825396826


In [248]:
prob = clf.predict_proba(test_X)

In [240]:
label_Y = clf.predict(test_X)
print(label_Y)

[0. 0. 1. ... 0. 0. 1.]


In [249]:
test_Y= prob[:,0]

In [250]:
predictions = open("predictions.csv", 'w')
predictions.write("ID,Pred\n")
k =0
for i,j in team_list:
    predictions.write('2019'+'_'+i+'_'+j+','+str('%.4f'%(test_Y[k]))+'\n')
    k=k+1

In [252]:
from sklearn.ensemble import RandomForestClassifier
tree_clf = RandomForestClassifier(n_estimators=100,random_state=0)
tree_clf.fit(train_X,train_Y)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=100, n_jobs=1,
            oob_score=False, random_state=0, verbose=0, warm_start=False)

In [258]:
print(tree_clf.feature_importances_)

[0.14380534 0.01272644 0.01693188 0.01027305 0.06027263 0.04360455
 0.00915474 0.05192739 0.08636355 0.01841547 0.01588243 0.00799176
 0.0272193  0.15275456 0.01256205 0.01628492 0.01008662 0.05999621
 0.04855271 0.00895298 0.04633821 0.06714368 0.01906085 0.01592123
 0.00870082 0.02907665]
